In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls ./drive/MyDrive/CS5242_project/personal_testing/data

CAM16_100cls_10mask  WBC_1  WBC_10  WBC_100  WBC_50


In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from tqdm import tqdm

# Define transformations for the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the WBC dataset
#data_dir = "./data/WBC_1/train/data"
data_dir = "./drive/MyDrive/CS5242_project/personal_testing/data/WBC_1/train/data"
#data_dir = "./drive/MyDrive/CS5242_project/personal_testing/data/WBC_10/train/data"
#data_dir = "./drive/MyDrive/CS5242_project/personal_testing/data/WBC_50/train/data"
#data_dir = "./drive/MyDrive/CS5242_project/personal_testing/data/WBC_100/train/data"
wbc_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
wbc_loader = DataLoader(wbc_dataset, batch_size=32, shuffle=True)

# Set device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the ResNet-18 model without pretraining
model = models.resnet18(pretrained=False).to(device)
num_classes = len(wbc_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Fetching Cam16 Dataset
cam16_data_dir = "./drive/MyDrive/CS5242_project/personal_testing/data/CAM16_100cls_10mask/train/data"
cam16_dataset = datasets.ImageFolder(root=cam16_data_dir, transform=transform)
cam16_loader = DataLoader(cam16_dataset, batch_size=32, shuffle=True)
model.fc = nn.Linear(model.fc.in_features, 2).to(device)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# """
# Training loop for CAM16 dataset
num_epochs_cam16 = 10  # You can adjust the number of epochs as needed
for epoch in range(num_epochs_cam16):
    model.train()  # Set the model to training mode
    total_loss = 0.0

    for inputs, labels in tqdm(cam16_loader, desc=f'Epoch {epoch + 1}/{num_epochs_cam16}'):
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)

        # Calculate loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(cam16_loader)
    print(f'Epoch [{epoch + 1}/{num_epochs_cam16}], Loss: {average_loss:.4f}')

#torch.save(model.state_dict(), "./drive/MyDrive/CS5242_project/personal_testing/pretrained_cam16_model.pth")

model.fc = nn.Linear(model.fc.in_features, num_classes).to(device)
# """
# Set up criterion (loss function) and optimizer
#criterion = nn.CrossEntropyLoss().to(device)
#optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

# Training loop
num_epochs = 10  # As needed

# Single-cycle learning rate strategy
total_iterations = len(wbc_loader) * num_epochs
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, total_steps=total_iterations)

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for inputs, labels in tqdm(wbc_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)

        # Calculate loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    average_loss = total_loss / len(wbc_loader)
    accuracy = (correct_predictions / total_samples) * 100
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}, Accuracy: {accuracy:.2f}%')

print("Training finished!")

# Define transformations for the evaluation dataset (without shuffling)
eval_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the WBC evaluation dataset
eval_data_dir = "./drive/MyDrive/CS5242_project/personal_testing/data/WBC_100/val/data"
#eval_data_dir = "./data/WBC_100/val/data"
eval_dataset = datasets.ImageFolder(root=eval_data_dir, transform=eval_transform)
eval_loader = DataLoader(eval_dataset, batch_size=32, shuffle=False, pin_memory=True)

# Save only the model weights
#torch.save(model.state_dict(), './drive/MyDrive/CS5242_project/personal_testing/resnet18_WBC1.pth')
#torch.save(model.state_dict(), './drive/MyDrive/CS5242_project/personal_testing/resnet18_WBC10.pth')
#torch.save(model.state_dict(), './drive/MyDrive/CS5242_project/personal_testing/resnet18_WBC50.pth')
torch.save(model.state_dict(), './drive/MyDrive/CS5242_project/personal_testing/resnet18_WBC100.pth')

# Calculate final accuracy on the evaluation dataset
model.eval()  # Set the model to evaluation mode
total_correct = 0
total_samples = 0

with torch.no_grad():
    for inputs, labels in tqdm(eval_loader, desc="Evaluating"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

final_accuracy = (total_correct / total_samples) * 100
print(f'Final Accuracy on the evaluation dataset: {final_accuracy:.2f}%')


Epoch 1/10: 100%|██████████| 24/24 [00:06<00:00,  3.78it/s]


Epoch [1/10], Loss: 0.6139


Epoch 2/10: 100%|██████████| 24/24 [00:06<00:00,  3.45it/s]


Epoch [2/10], Loss: 0.3447


Epoch 3/10: 100%|██████████| 24/24 [00:06<00:00,  3.84it/s]


...

Epoch [9/10], Loss: 0.0564, Accuracy: 98.18%


Epoch 10/10: 100%|██████████| 264/264 [01:21<00:00,  3.24it/s]


Epoch [10/10], Loss: 0.0423, Accuracy: 98.63%
Training finished!


Evaluating: 100%|██████████| 54/54 [00:13<00:00,  3.96it/s]

Final Accuracy on the evaluation dataset: 98.03%
